In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#pd.options.plotting.backend = "plotly"

#import logging
# from pyspedas.cotrans.cotrans_get_coord import cotrans_get_coord
# from pyspedas.cotrans.cotrans import cotrans
# from pyspedas.cotrans.gsm2lmn import gsm2lmn
from pyspedas.mms import mec
#from pyspedas import tinterpol, omni
import os
import pyspedas
from pytplot import tplot, get_data, store_data, tlimit, options,tplot_names
from pyspedas.mms import fpi,fgm
#from pyspedas.mms.mms_orbit_plot import mms_orbit_plot

# logging.captureWarnings(True)
# logging.basicConfig(format='%(asctime)s: %(message)s', datefmt='%d-%b-%y %H:%M:%S', level=logging.INFO)


In [2]:

MMS2_B = pd.read_csv('MMS2_B.csv',header = 0,names = ['times','Bx','By','Bz','Bmag','date'],
                     parse_dates=['date'])


MMS2_V = pd.read_csv('MMS2_V.csv',
                     header = 0,names = ['times','Vx','Vy','Vz','date'],
                     parse_dates=['date'])

MMS2_n = pd.read_csv('MMS2_n.csv',
                     header = 0,names = ['times','n','date'],
                     parse_dates=['date'])

MMS2_Tperp = pd.read_csv('MMS2_Tperp.csv',
                     header = 0,names = ['times','Tperp','date'],
                     parse_dates=['date'])

MMS2_Tpara = pd.read_csv('MMS2_Tpara.csv',
                     header = 0,names = ['times','Tpara','date'],
                     parse_dates=['date'])




In [3]:
MMS2_B

,times,Bx,By,Bz,Bmag,date
0,1.592426e+09,6.831184,2.097465,8.011147,10.735126,2020-06-17 20:29:23.249434112
1,1.592426e+09,6.841014,2.092012,8.037822,10.760231,2020-06-17 20:29:23.257246976
2,1.592426e+09,6.823400,2.076468,8.044454,10.750989,2020-06-17 20:29:23.265059072
3,1.592426e+09,6.798868,2.074919,8.046974,10.737024,2020-06-17 20:29:23.272871936
4,1.592426e+09,6.819839,2.061433,8.057381,10.755514,2020-06-17 20:29:23.280684032
...,...,...,...,...,...,...
2308347,1.592444e+09,2.217578,2.958381,8.717915,9.469515,2020-06-18 01:30:02.413986048
2308348,1.592444e+09,2.205297,2.972957,8.730649,9.482935,2020-06-18 01:30:02.421797888
2308349,1.592444e+09,2.226805,2.989825,8.726459,9.489406,2020-06-18 01:30:02.429611008
2308350,1.592444e+09,2.229691,2.994750,8.727658,9.492738,2020-06-18 01:30:02.437423104


In [4]:
# supply the current sheet start time and end time:
CS_st_t = '2020-06-17/21:54:23'


CS_end_t = '2020-06-17/21:54:35'


# time period before CS to take average of B-field:
st_ave_B1 = '2020-06-17/21:54:20'
end_ave_B1 = '2020-06-17/21:54:23'

st_ave_B2 = '2020-06-17/21:54:35'
end_ave_B2 = '2020-06-17/21:54:38'





st_idx_B1 = MMS2_B.date.searchsorted(st_ave_B1)
end_idx_B1 = MMS2_B.date.searchsorted(end_ave_B1)




st_idx_B2 = MMS2_B.date.searchsorted(st_ave_B2)
end_idx_B2 = MMS2_B.date.searchsorted(end_ave_B2)


print(st_idx_B1)
print(end_idx_B1)

Bz1 = np.array(pd.to_numeric(MMS2_B['Bz']))
ave_B1 = np.mean(Bz1[st_idx_B1:end_idx_B1])

print(ave_B1)
Bz2 = np.array(pd.to_numeric(MMS2_B['Bz']))
ave_B2 = np.mean(Bz2[st_idx_B2:end_idx_B2])

print(ave_B2)





#     end_idx_B = MMS2_B.date.searchsorted(end_time)

652377
652761
6.2794053335937505
7.2512223664062505


In [5]:
from numpy import linalg as LA
'''
Take in magnetic field dataframe mag_arr e.g., 'MMS2_b.csv'
Take in specified start time (m1) of current sheet e.g., '2020-06-17/21:54:20'

take in specified end time (m2) of current sheet crossing e.g., '2020-06-17/21:54:23'

Code from Eriksson

'''



# In summary, the minimum variance analysis consists of constructing the matrix M_{μν}^{B}, 
# defined by equation 8.8 in terms of the measured field data and the cartesian coordinate 
# system in which the measured data are represented, and then finding the three eigenvalues λi, 
# and corresponding eigenvectors xi, of the matrix. The eigenvector x3 corresponding to the 
# smallest eigenvalue, λ3, is used as the estimator for the vector normal to the current sheet 
# and λ3 itself represents the variance of the magnetic field component along the estimated normal. 
# The eigenvectors x1 and x2, corresponding to maximum and intermediate variance, are then tangential 
# to the transition layer and the set {x1,x2,x3} arranged as a right-handed orthonormal triad provides 
# suitable basis vectors for the local coordinates (x,y,z) discussed in connection with equation 8.1.

# #This function performs the minimum variance analysis for
# #the magnetic field between times m1 and m2 to get an L,M,N
# #coordinate system
# # ex. m1='2001-02-04/21:40:00'
# #
# # reference: Sonnerup, B. U. O., and M. Scheible, Minimum and maximum variance
# #	     analysis, in Analysis methods for multi-spacecraft data, edited by
# #	     G. Paschmann and P. W. Daly, ISSI Sci. Rep., SR-001, p. 185-220,
# #	     ESA Publications Division, Noordwijk, The Netherlands, 1998.
# #	     (see ~eriksson/papers/issi_analysis_methods.pdf)

# #########################
# #GET MAGNETIC FIELD DATA#
# #########################







def mms_mvab(m1,m2,mag_arr_str):
    mag_arr = pd.read_csv(mag_arr_str,header = 0,names = ['times','Bx','By','Bz','Bmag','date'],
                         parse_dates=['date'])
    
    m1 = pd.to_datetime(m1)
    m2 = pd.to_datetime(m2)
    
    # ##########################################################
    # Check for NaN values
    NaNs = mag_arr.isnull().values.any()

    if NaNs == True:
        print,'Nans were found: '
        print, mag_arr.isnull().values.any()
        print,'Replacing with average of surrounding data pts '
        # Indices of NaNs in each component
        NaNs_Bx = np.argwhere(np.isnan(bx))
        NaNs_By = np.argwhere(np.isnan(by))
        NaNs_Bz = np.argwhere(np.isnan(bz))

        # If there are NaNs, replace with the average of surrounding couple of data points
        for i in range(len(NaNs_Bx)):
            bx[NaNs_Bx[i]] = np.nanmean(bx[NaNs_Bx[i-3]:NaNs_Bx[i+3]])
            by[NaNs_By[i]] = np.nanmean(by[NaNs_By[i-3]:NaNs_By[i+3]])
            bz[NaNs_Bz[i]] = np.nanmean(bz[NaNs_Bz[i-3]:NaNs_Bz[i+3]])                           
 
    time = np.array(mag_arr['times'])
    bx = np.array(pd.to_numeric(mag_arr['Bx']))
    by = np.array(pd.to_numeric(mag_arr['By']))
    bz = np.array(pd.to_numeric(mag_arr['Bz']))
    
    
    # start and stop times of the magnetic field data from the .csv
    n1 = mag_arr.date[0]
    n2 = mag_arr.date[len(mag_arr.date)-1]

     # Check if the start time is outside of the magnetic field timestamps by more than an hour
    if m1 < n1:
        print('WARNING!!! start time of current sheet =',m1, 'start time of B-field data =', n1)
    if m1 > n2:
        print('WARNING!!! start time of current sheet =',m1,'end time of B-field data =',n2)
    if m2 > n2:
        print('WARNING!!! end time of current sheet =',m2,'end time of B-field data =',n2)
    if m2 < n1:
        print('WARNING!!! end time of current sheet =',m2,'start time of B-field data =',n1)
    
    # search the B-field array date column for m1
    v1 = mag_arr.date.searchsorted(m1)
    v2 = mag_arr.date.searchsorted(m2)
    
    # as of 2024-03-28 if the value isn't in the array, searchsorted returns the closest value,
    # e.g., if your looking for '2020-06-03/13:00:00' in an array that spans 
    # 2020-06-17/20:30:00 - 2020-06-18/01:30:00', searchsorted returns 0. If e.g., you're looking for
    # '2024-06-03/13:00:00' it would return the last index in the array. Have to safe guard against this.
    # Maybe specify if the element is more than an hour away from target, then the 
    # timestamp can't be found in the array.

    bx = bx[v1:v2]
    by = by[v1:v2]
    bz = bz[v1:v2]

    # ###################################
    # # GET the Magnetic Variance Matrix#
    # # eq. 8.8, p. 188 in reference    #
    # ###################################


    # magnetic variance matrix                                    
    mm = np.zeros([3, 3])

    bxa = np.mean(bx)
    bya = np.mean(by)
    bza = np.mean(bz)

    print('0th moment of bx:',bxa)
    print('0th moment of by:',bya)
    print('0th moment of bz:',bza)

    # #1st row
    mm[0,0] = np.mean(bx*bx) - bxa*bxa
    mm[1,0] = np.mean(bx*by) - bxa*bya
    mm[2,0] = np.mean(bx*bz) - bxa*bza

    # #2nd row
    mm[0,1] = mm[1,0]
    mm[1,1] = np.mean(by*by) - bya*bya
    mm[2,1] = np.mean(by*bz) - bya*bza

    # #3rd row
    mm[0,2] = mm[2,0]
    mm[1,2] = mm[2,1]
    mm[2,2] = np.mean(bz*bz) - bza*bza

    print('\n')
    print('\n')
    print('mm:',mm)

    ##################################################################
    ## GET EIGENVALUES and Eigenvectors of Magnetic Variance Matrix ##
    ##################################################################
    eigenvalues, eigenvectors = LA.eig(mm)    


    idx = np.argsort(eigenvalues)
    E_vals_sorted = eigenvalues[idx]
    E_vec_sorted = eigenvectors[:,idx]

    print('\n')
    print('\n')
    print('eigenvalues:')
    print(E_vals_sorted)
    print('eigenvectors:')                                   
    print(E_vec_sorted)

    # The eigenvector x1 corresponding to the smallest eigenvalue, λ1, 
    # is used as the estimator for the vector normal to the current sheet 
    # and λ1 itself represents the variance of the magnetic field component along the estimated normal.

    # λ1's eigenvector is the vector normal to the CS.


    # max variance -> λ3 -> L_hat* = x3

    return E_vals_sorted,E_vec_sorted


In [6]:
m1 = '2020-06-17/21:54:20'
m2 = '2020-06-17/21:54:38'
mag_arr_str = 'MMS2_B.csv'
E_vals,E_vec = mms_mvab(m1,m2,mag_arr_str)

0th moment of bx: 7.859658329687499
0th moment of by: 1.482304751860113
0th moment of bz: 8.310890515625




mm: [[ 4.5157527   3.59586811 -2.87233924]
 [ 3.59586811  3.10864317 -2.48166863]
 [-2.87233924 -2.48166863  2.13095602]]




eigenvalues:
[0.07462275 0.23540039 9.44532875]
eigenvectors:
[[-0.23083367 -0.69356434 -0.68241067]
 [ 0.7651781   0.30382417 -0.56762078]
 [ 0.60101439 -0.65319168  0.4605674 ]]


In [29]:
# Below are some lines of IDL code to get the cross-product normal.

def Cross_product_normal(E_vals,E_vec,m1,m2,mag_arr_str):

    mag_arr = pd.read_csv(mag_arr_str,header = 0,names = ['times','Bx','By','Bz','Bmag','date'],
                         parse_dates=['date'])

    m1 = pd.to_datetime(m1)
    m2 = pd.to_datetime(m2)


    # add check point to make sure eigenvectors are appropriate length/values

    nv = E_vec[:,0]
    #print('nv:',nv)
    lv = E_vec[:,2]
    #print('lv',lv)
    mv = np.cross(nv,lv)

    

    rat = E_vals[1]/E_vals[0]

    lxm = lv[0]
    lym = lv[1]
    lzm = lv[2]

    mxm = mv[0]
    mym = mv[1]
    mzm = mv[2]

    nxm = nv[0]
    nym = nv[1]
    nzm = nv[2]


    # ##########################################################
    # #Cross-Product Normal - get difference angles to MVAB LMN#
    # ##########################################################

    time = np.array(mag_arr['times'])
    bx = np.array(pd.to_numeric(mag_arr['Bx']))
    by = np.array(pd.to_numeric(mag_arr['By']))
    bz = np.array(pd.to_numeric(mag_arr['Bz']))



    # search the B-field array date column for m1
    v1 = mag_arr.date.searchsorted(m1)
    v2 = mag_arr.date.searchsorted(m2)


    dtb = pd.Timedelta(hours = 0, minutes = 0, seconds = 2)


    v1a = mag_arr.date.searchsorted(m1 - dtb)
    v2a = mag_arr.date.searchsorted(m2 + dtb)

    bv1x = bx[v1a:v1]
    bv1y = by[v1a:v1]
    bv1z = bz[v1a:v1]

    bv2x = bx[v2:v2a]
    bv2y = by[v2:v2a]
    bv2z = bz[v2:v2a]
    


    bv1avg = [np.mean(bv1x),np.mean(bv1y),np.mean(bv1z)]

    bv2avg = [np.mean(bv2x),np.mean(bv2y),np.mean(bv2z)]



    bmag1 = np.sqrt(np.sum(np.multiply(bv1avg,bv1avg)))
    bmag2 = np.sqrt(np.sum(np.multiply(bv2avg,bv2avg)))
    #bshearcp = np.acos(sum(bv1avg*bv2avg)/(bmag1*bmag2))*180./!dpi
    bshearcp = np.arccos(np.sum(np.multiply(bv1avg,bv2avg))/(bmag1*bmag2))*180



    normal = np.cross(bv1avg,bv2avg)
    normag = np.sqrt(np.sum(np.multiply(normal,normal)))
    nc = normal/normag  # N-direction

    # #Angle between cross-product & MVAB normals
    #n12ang = np.arccos(np.sum(nv*nc))*180./!dpi
    n12ang = np.arccos(np.sum(nv*nc))*180

    

    if n12ang >= 90.0:
        nc = -nc

    mvec = np.cross(nc,E_vec[:,2])
    mmag = np.sqrt(np.sum(np.multiply(mvec,mvec)))
    mc = mvec/mmag    #M-direction

    lc = np.cross(mc,nc)  #L-direction
    angle = np.arccos(np.sum(np.multiply(E_vec[:,2],lc)))*180



    # Angle between cross-product & MVAB normals
    n12ang = np.arccos(np.sum(np.multiply(nv,nc)))*180

    
    ###################################################
    ## Use these print statements to make sure things are working correctly
    ##
    ###################################################
    
#     print('start time of current sheet:',m1,'\n')
#     print('end time of current sheet:',m2,'\n')
#     print('MVAB of N (GSE)',nv,'\n')

#     print('MVAB of L (GSE)',lv,'\n')
#     print('MVAB of M (GSE)',mv,'\n')
#     # average B-components from start_time - dtb to start time
#     print('bv1avg',bv1avg,'\n')
#     print('Bgse1 (nT):',bv1avg,'\n')
#     print('Bgse2 (nT):',bv2avg,'\n')
#     print('Bshear (deg):',bshearcp,'\n')
#     # #Angle between cross-product & MVAB normals
#     print('Nv*Nc =',n12ang,'\n')
#     # N-direction components?
#     print('Ncp_gse: ',nc[0],nc[1],nc[2],'\n')
#     # L-direction components?
#     print('Lcp_gse: ',lc[0],lc[1],lc[2],'\n')
#     # M-direction components? But not sure why they're in GSE
#     print('Mcp_gse: ',mc[0],mc[1],mc[2],'\n')
    
#     # print angle between cross-product &MVAB normals
#     print('Nv*Nc =',n12ang,'\n')

    #bs2=bshear
    lxc = lc[0]
    lyc = lc[1]
    lzc = lc[2]
    mxc = mc[0]
    myc = mc[1]
    mzc = mc[2]
    nxc = nc[0]
    nyc = nc[1]
    nzc = nc[2]
    nang = n12ang

    return nc,mc,lc

In [30]:
nc,mc,lc = Cross_product_normal(E_vals,E_vec,m1,m2,mag_arr_str)

In [31]:
# make sure the dot products are ~0

print(np.dot(nc,lc))
print(np.dot(mc,lc))
print(np.dot(nc,mc))

3.8567945301000445e-17
1.2384516474336108e-17
7.506839865497363e-18


In [ ]:



# Using identification criteria from Eriksson et al., 2016
# Vx jet candidates
jet_candidate_timeranges = ['2020-06-17/22:01:38','2020-06-17/22:01:46']

# 2020-06-17/22:16:55 - 2020-06-17/22:17:04
# 2020-06-18/00:06:57 - 2020-06-18/00:07:02
# 2020-06-18/00:15:36 - 2020-06-18/00:15:40 


# # Vy jet candidates
# 2020-06-17/21:35:56 - 2020-06-17/21:36:04
# 2020-06-17/22:00:47 - 2020-06017/22:00:55 

# # Vz jet candidates
# 2020-06-17/21:16:35 - 2020-06-17/21:16:41

# 2020-06-17/21:16:57 - 2020-06-17/21:17:05

# 2020-06-17/21:42:47 - 2020-06-17/21:42:54 (Vz, possibly vy)

# 2020-06-17/21:47:19 - 2020-06-17/21:47:30 ( Very asymmetric)

# 2020-06-17/21:54:23 - 2020-06-17/21:54:25 

# 2020-06-17/23:42:48 - 2020-06-17/23:42:57 (Possibly Vx candidate as well)






nc_mag = np.sqrt((np.sum(np.multiply(nc,nc))))

lc_mag = np.sqrt((np.sum(np.multiply(lc,lc))))

mc_mag = np.sqrt((np.sum(np.multiply(mc,mc))))

# make sure the magnitudes are ~1

print(nc_mag)
print(lc_mag)
print(mc_mag)







bn = []
bl = []
bm = []

mag_arr = pd.read_csv(mag_arr_str,header = 0,names = ['times','Bx','By','Bz','Bmag','date'],
                         parse_dates=['date'])

m1 = pd.to_datetime(jet_candidate_timeranges[0])
m2 = pd.to_datetime(jet_candidate_timeranges[1])

time = np.array(mag_arr['times'])
bx = np.array(pd.to_numeric(mag_arr['Bx']))
by = np.array(pd.to_numeric(mag_arr['By']))
bz = np.array(pd.to_numeric(mag_arr['Bz']))



# search the B-field array date column for m1
v1 = mag_arr.date.searchsorted(m1)
v2 = mag_arr.date.searchsorted(m2)

bvx = bx[v1:v2]
bvy = by[v1:v2]
bvz = bz[v1:v2]
time1 = time[v1:v2]



print('number of data points to be examined:',len(time1),'\n')
# for each time stamp in the B-field from start and stop time of jet, find the new LMN components
for t in range(len(time1)):
    
    # calculate cross product normals using eigen values, eigen vectors, and the start and stop 
    # time for each jet candidate. 
    nc,mc,lc = Cross_product_normal(E_vals,E_vec,jet_candidate_timeranges[0],
                                          jet_candidate_timeranges[1],mag_arr_str)
    
    bvx_temp = bvx[t]
    bvy_temp = bvy[t]
    bvz_temp = bvz[t]
    # create matrix where columns are the x,y,z component at that timestamp to be used in the following dot product
    bvec = np.stack((bvx_temp,bvy_temp,bvz_temp), axis=-1)
    
    # find dot product between bvec and nc
    bl.append(np.dot(bvec,lc))
    bm.append(np.dot(bvec,mc))
    bn.append(np.dot(bvec,nc))


# print(bvec.ndim)
# print(bvec.shape)
# bn = np.dot(bvec,nc)
# bl = np.dot(bvec,lc)
# bm = np.dot(bvec,mc)

fig = plt.figure(figsize=(10,10))
plt.plot(time1,bl)
plt.plot(time1,bm)
plt.plot(time1,bn)



plt.show()

1.0
1.0
1.0


In [ ]:
print('$B_L$',bl,'\n')
print('$B_M$',bm,'\n')
print('$B_N$',bn,'\n')

In [ ]:
# Matplotlib plotting
%matplotlib widget


fig = plt.figure(figsize=(10,10))
plt.rcParams.update({'font.size': 12})



# Bx GSE
ax_Bx = fig.add_subplot(911)
#ax_Bx.set_title('Raw Data')
#ax_Bx.plot(MMS2_B_trimmed['date'],MMS2_B_trimmed['Bx'], label='$B_x$', color ='black')
ax_Bx.plot(MMS2_B['date'],MMS2_B['Bx'], label='$B_x$', color ='black')
ax_Bx.set_ylabel('$B_x$ (GSE) \n [nT]')
ax_Bx.grid('on')
#ax_Bx.set_xticklabels([])
#ax_Bx.set_ylim([-50,50])

#Vx GSE
ax_Vx = fig.add_subplot(912,sharex = ax_Bx)
#ax_Vx.set_title('Raw Data')
ax_Vx.plot(MMS2_V['date'],MMS2_V['Vx'], label='$V_x$', color ='black')
ax_Vx.set_ylabel('$V_x$ (GSE) \n [km s$^{-1}$]')
ax_Vx.grid('on')
ax_Vx.set_ylim([-75,75])



# By GSE
ax_By = fig.add_subplot(913,sharex = ax_Bx)
#ax_By.plot(MMS2_B_trimmed['date'],MMS2_B_trimmed['By'], label='$B_y$', color ='darkturquoise')
ax_By.plot(MMS2_B['date'],MMS2_B['By'], label='$B_y$', color ='darkturquoise')
ax_By.set_ylabel('$B_y$ (GSE) \n [nT]')
ax_By.grid('on')
#ax_By.set_ylim([-50,50])

#Vy GSE
ax_Vy = fig.add_subplot(914,sharex = ax_Bx)
ax_Vy.plot(MMS2_V['date'],MMS2_V['Vy'], label='$V_y$', color ='darkturquoise')
ax_Vy.set_ylabel('$V_y$ (GSE) \n [km s$^{-1}$]')
ax_Vy.grid('on')
#ax_Vy.set_ylim([-50,50]



#Bz GSE
ax_Bz = fig.add_subplot(915,sharex = ax_Bx)
#ax_Bz.plot(MMS2_B_trimmed['date'],MMS2_B_trimmed['Bz'], label='$B_z$', color ='rebeccapurple')
ax_Bz.plot(MMS2_B['date'],MMS2_B['Bz'], label='$B_z$', color ='rebeccapurple')
ax_Bz.set_ylabel('$B_z$ (GSE) \n [nT]')
ax_Bz.grid('on')
#ax_Bz.set_ylim([-50,50])




#Vz GSE
ax_Vz = fig.add_subplot(916,sharex = ax_Bx)
ax_Vz.plot(MMS2_V['date'],MMS2_V['Vz'], label='$V_z$', color ='rebeccapurple')
ax_Vz.set_ylabel('$V_z$ (GSE) \n [km s$^{-1}$]')
ax_Vz.grid('on')

# ion density
ax_n = fig.add_subplot(917,sharex = ax_Bx)
ax_n.plot(MMS2_n['date'],MMS2_n['n'], label='$n$',color = 'maroon')
ax_n.set_ylabel('n \n [cm$^{-3}$]')
ax_n.grid('on')
#ax_n.set_xticklabels([])



# ion parallel and perpendicular temperature
ax_T = fig.add_subplot(918,sharex = ax_Bx)
ax_T.plot(MMS2_Tperp['date'],MMS2_Tperp['Tperp'], label='$T_{⊥}$',color ='plum')
ax_T.plot(MMS2_Tpara['date'],MMS2_Tpara['Tpara'], label='$T_{||}$',color ='seagreen')
ax_T.set_ylabel('$T$ \n [eV]')
ax_T.grid('on')
ax_T.legend(loc='center right',fontsize='small')



# ax_IMF = fig.add_subplot(918,sharex = ax_Bx)
# ax_IMF.plot(OMNI_IMF['date'],OMNI_IMF['IMF'], label='IMF',color ='darkgray')
# #ax_IMF.plot(OMNI_flow_['date'],OMNI_IMF['IMF'], label='$T_{||}$',color ='seagreen')
# ax_IMF.set_ylabel('IMF ')
# ax_IMF.grid('on')
# #x_IMF.legend(loc='center right',fontsize='small')




# #ax1.set_xticks([])
# plt.grid('on')
# plt.legend()
# ax2 = fig.add_subplot(512)
# ax2.plot(MMS2_n['date'],MMS2_n['n'], label='$n$')
# ax2.set_ylabel('n \n [cm$^{-3}$]')
# ax2.grid('on')
# #ax2.set_xticks([])
# #plt.grid('on')
# ax3 = fig.add_subplot(513)
# ax3.plot(MMS2_Tpara['date'],MMS2_Tpara['Tpara'], label='$T_{||}$')
# ax3.set_ylabel('$T_{||}$ \n [eV]')
# ax3.grid('on')
# #ax3.set_xticks([])
# #plt.grid('on')
# ax4 = fig.add_subplot(514)
# ax4.plot(MMS2_Tperp['date'],MMS2_Tperp['Tperp'], label='$T_{⊥}$')
# ax4.set_ylabel('$T_{⊥}$ \n [eV]')
# ax4.grid('on')
# #ax4.set_xticks([])
# #plt.grid('on')
# ax5 = fig.add_subplot(515)
# ax5.plot(MMS2_V['date'],MMS2_V['Vx'], label='$V_x$')
# ax5.plot(MMS2_V['date'],MMS2_V['Vy'], label='$V_y$')
# ax5.plot(MMS2_V['date'],MMS2_V['Vz'], label='$V_z$')
# ax5.set_ylabel('$V}$ \n [km s$^{-1}$]')
# ax5.grid('on')

plt.subplots_adjust(hspace=0.0)

#plt.legend()
#plt.grid('on')

plt.show()
# plt.close()
#fig.savefig('Full_time_period_data')